In [6]:
import pickle
import transformers
import torch
import pandas as pd

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU available')
else:
    device = torch.device('cpu')
    print('Uh oh, GPU unavailable')

GPU available


In [2]:
with open('election_dataset_raw.pickle', 'rb') as f:
    biden_tweets = pickle.load(f)
biden = biden_tweets['biden']
biden_df = pd.DataFrame(biden, columns = ["datetime", "text"])

In [9]:
mid = len(biden_df) // 2
biden_first = biden[0:mid]
biden_last = biden[mid:]

In [10]:
pickle.dump(biden_first, open("biden_first_half.pickle", 'wb'))
pickle.dump(biden_last, open("biden_last_half.pickle", 'wb'))